In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import random
import os
import gc
from PIL import Image
from textwrap import wrap
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from tensorflow.python.keras import models, layers, optimizers
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding
from keras.layers import GlobalMaxPooling1D
from keras.layers.core import Activation, Dropout, Dense
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import bz2
import re

%matplotlib inline


In [4]:
df=pd.read_csv('/content/drive/MyDrive/irfinalproject/file.csv')

In [5]:
from sklearn.model_selection import train_test_split
train_labels, test_labels, train_texts , test_texts = train_test_split(df['Label'],df['Text'] ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

In [6]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):

    normalized_texts = []
    
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [7]:
train_texts_Bert = train_texts
train_labels_Bert = train_labels
test_texts_Bert = test_texts
test_labels_Bert = test_labels
train_reviews = train_texts

In [8]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=57643892, test_size=0.2)

In [9]:
MAX_FEATURES = 12000
#Tokenize texts
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

In [10]:
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
#Add padding
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
val_texts = pad_sequences(val_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)

In [11]:
def build_lstm_model():
    sequences = layers.Input(shape=(MAX_LENGTH,))
    embedded = layers.Embedding(MAX_FEATURES, 64)(sequences)
    x = layers.LSTM(32, return_sequences=True)(embedded)
    x = layers.GlobalMaxPool1D()(x)
    x = layers.Dense(16, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    predictions = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs=sequences, outputs=predictions)
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model
    
lstm_model = build_lstm_model()

In [12]:
print(lstm_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 868)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 868, 64)           768000    
_________________________________________________________________
lstm (LSTM)                  (None, 868, 32)           12416     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17    

In [13]:
lstm_model.fit(
    train_texts, 
    train_labels, 
    batch_size=128,
    epochs=2,
    validation_data=(val_texts, val_labels), )

Epoch 1/2
235/235 [==============================] - 334s 1s/step - loss: 0.4930 - accuracy: 0.7504 - val_loss: 0.3192 - val_accuracy: 0.8657
Epoch 2/2
235/235 [==============================] - 323s 1s/step - loss: 0.2703 - accuracy: 0.8933 - val_loss: 0.3061 - val_accuracy: 0.8739


In [14]:
preds = lstm_model.predict(test_texts)
print('Accuracy score: {:0.4}'.format(accuracy_score(test_labels, 1 * (preds > 0.5))))
print('F1 score: {:0.4}'.format(f1_score(test_labels, 1 * (preds > 0.5))))
print('ROC AUC score: {:0.4}'.format(roc_auc_score(test_labels, preds)))

Accuracy score: 0.8735
F1 score: 0.8909
ROC AUC score: 0.9423


In [15]:
lstm_model.save_weights('/content/drive/MyDrive/irfinalproject/lstm.h5')

In [16]:
model1 = build_lstm_model()
model1.load_weights('/content/drive/MyDrive/irfinalproject/lstm.h5')

In [17]:
input_text = "Good product, must buy !!"
input_text = tokenizer.texts_to_sequences([input_text])
input_text = pad_sequences(input_text, maxlen=MAX_LENGTH)
#prediction = lstm_model.predict(input_text)
prediction = model1.predict(input_text)
label = 1 if prediction > 0.5 else 0
print(label)

1
